In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("OPENAI_ENDPOINT")
model = "gpt-4o-mini"
temperature=0.0

llm = ChatOpenAI(
    base_url=base_url,
    api_key=api_key,
    model=model,
    temperature=temperature
)

## **Understanding LangChain llm.invoke() method**
- **invoke()** can take basically **anything that has to_string() method** and gives a prompt string
  - This includes simple string, , dictionary, list of strings, list of dictionaries, and StringPromptValue object, ..etc.
- **invoke()** returns **AIMessage** object that has a **content** string attribute 

In [ ]:
llm.invoke("Hello there") # returns AIMessage object

In [ ]:
response = llm.invoke("Hi There")

print(response.content) # content str only

In [ ]:
response = llm.invoke("Hi There")

print(type(response))  # AIMessage

print(response.content) # content str only
print(response) # AIMessage object


In [ ]:
message_list = ["Hello There", "How are you?"]

response = llm.invoke(message_list)

print(response.content)

In [ ]:
message_list = [
        {"role": "user", "content": "Hello There"},
        {"role": "user", "content": "How are you?"},
    ]
response = llm.invoke(message_list)

print(response.content)


## **Chat History**
- A list of LangChain message objects: SystemMessage, AIMessage, HumanMessage, etc.

In [ ]:
messages = [ 
    SystemMessage("You are a geography tutor"),
    HumanMessage("What's the capital of Brazil?")
]


In [ ]:
# Invoke can take string, AIMessage, or a list of AIMessages
response = llm.invoke(messages)
print(response.content)

In [ ]:
messages = [ 
    SystemMessage("You are a geography tutor"),
    HumanMessage("What's the capital of Brazil?"),
    AIMessage("The capital of Brazil is Brasília"),
    HumanMessage("What's the capital of Canada?"),
]

In [ ]:
# Here the messages included kind of training by providing example in the chat history
response = llm.invoke(messages)
print(response.content)

## **Prompting with Python String**
- Using mainly f-strings and str.format()

In [ ]:
# using f-strings # variables has to be assigned first
topic = "Python"
prompt = f"Tell me a joke about {topic}"
response = llm.invoke(prompt)
print(response.content)

In [ ]:
# using str.format() # variables can be assigned later
prompt = "Tell me a joke about {subject}"
response = llm.invoke(prompt.format(subject) = "Python"))
print(response.content)

## **PromptTemplate object**
- format() vs. Invoke()


In [ ]:
prompt_template = PromptTemplate(
    template="Tell me a joke about {var1} as long as it's not {var2} "
)

In [ ]:
prompt_template

In [ ]:
# prompt format() returns string and  can take keywords as arguments
prompt_template.format(var1="Python", var2="offensive")

In [ ]:
# prompt invoke returns StringPromptValue object and takes a dictionary of input variables and their values
prompt_template.invoke({"var1":"Python","var2":"offensive"})

In [ ]:
response = llm.invoke(prompt_template.invoke({"var1":"Python","var2":"offensive"}))
print(response.content)

## **Few Shot Prompt**

In [ ]:
example_prompt = PromptTemplate(
    template="Question: {input}\nThought: {thought}\nResponse: {output}"
)

In [ ]:
examples = [
    {
        "input": "A train leaves city A for city B at 60 km/h, and another train leaves city B for city A at 40 km/h. If the distance between them is 200 km, how long until they meet?", 
        "thought": "The trains are moving towards each other, so their relative speed is 60 + 40 = 100 km/h. The time to meet is distance divided by relative speed: 200 / 100 = 2 hours.",
        "output": "2 hours",
    },
    {
        "input": "If a store applies a 20% discount to a $50 item, what is the final price?", 
        "thought": "A 20% discount means multiplying by 0.8. So, $50 × 0.8 = $40.",
        "output": "$40",
    },
    {
        "input": "A farmer has chickens and cows. If there are 10 heads and 32 legs, how many of each animal are there?", 
        "thought": "Let x be chickens and y be cows. We have two equations: x + y = 10 (heads) and 2x + 4y = 32 (legs). Solving: x + y = 10 → x = 10 - y. Substituting: 2(10 - y) + 4y = 32 → 20 - 2y + 4y = 32 → 2y = 12 → y = 6, so x = 4.",
        "output": "4 chickens, 6 cows",
    },
    {
        "input": "If a car travels 90 km in 1.5 hours, what is its average speed?", 
        "thought": "Speed is distance divided by time: 90 km / 1.5 hours = 60 km/h.",
        "output": "60 km/h",
    },
    {
        "input": "John is twice as old as Alice. In 5 years, their combined age will be 35. How old is Alice now?", 
        "thought": "Let Alice's age be x. Then John’s age is 2x. In 5 years, their ages will be x+5 and 2x+5. Their sum is 35: x+5 + 2x+5 = 35 → 3x + 10 = 35 → 3x = 25 → x = 8.33.",
        "output": "8.33 years old",
    },
]

In [ ]:
example_prompt.invoke(examples[0]).to_string()

In [ ]:
# This is how you create few shot prompt in LangChain
prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt_template.format(input="If today is Wednesday, what day will it be in 10 days?"))
print(prompt_template.invoke({"input":"If today is Wednesday, what day will it be in 10 days?"}))

In [ ]:
response = llm.invoke(
    prompt_template.format(input="If today is Wednesday, what day will it be in 10 days?")
)
print(response.content) # Response should be Saturday

In [ ]:
response = llm.invoke(
    prompt_template.invoke({"input":"If today is Wednesday, what day will it be in 10 days?"})
)
print(response.content) # Response should be Saturday